# Auxiliar 1 


----------------------------------

## 📚 Objetivos de la clase 📚

El objetivo principal de esta clase es introducirlos a la clasificación de texto en NLP. 
Para esto, implementaremos varios modelos de clasificación destinados a **predecir la categoría de noticias de la radio biobio**.

Los modelos y métodos que usaremos serán las vistas en las clases anteriores: 

- Preprocesamiento: Tokenización, Stemming, Lematización y eliminación de Stop Words.
- Bag of Words.
- Claisifcador de Bayes .
- Logistic regression.

La clase estará enfocada en utilizar las siguientes librerías (muy utilizadas en NLP):

- Pandas
- Scikit-Learn
- Spacy
- NLTK

Una vez resuelto, pueden utilizar cualquier parte del código que les parezca prudente para la tarea 1 (que también es de clasificación de texto! 😊).



El notebook del auxiliar ya ejecutado se encuentra tanto en el [github](https://github.com/dccuchile/CC6205/tree/master/tutorials) del curso (Recuerden dejar su Star ⭐😉!), como en un colab de google. 


TODO: Agregar referencias al video y a las diapos aquí:

Para correr localmente el auxiliar, se recomienda instalar todos los paquetes usando anaconda,

## Importar las librerías

### En local: Python y Conda

Primero, si es que no tienen aun las librerías, hay que instalarlas.
Recuerden que usaremos `python 3.7` junto a `conda` como gestor de paquetes para el curso.

Este pueden descargarlo e instalenlo desde aquí : [🐍 Anaconda 🐍](https://www.anaconda.com/distribution/).

Para instalar las librerías, ejecutar en una consola 💻:

```cmd
conda install pandas scikit-learn ntkl spacy 
```
Y luego descargar el modelo de spacy en español: 

```cmd
python -m spacy download es_core_news_sm
```

Si saben un poco mas de anaconda, pueden instalar sus paquetes en un ambiente exlcusivo para el curso. Pero no es necesario!! Mas información [aquí](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html).

### En colab

TODO


### Importar

In [8]:
import pandas as pd    
import spacy
import nltk

In [9]:
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
from nltk.stem import SnowballStemmer
from spacy.lang.es.stop_words import STOP_WORDS
nlp = spacy.load("es_core_news_sm", disable=['ner', 'parser', 'tagger'])
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Clasificación de Texto 

¿Cuál de estos emails es SPAM?

![respuesta spam](https://raw.githubusercontent.com/dccuchile/CC6205/master/tutorials/recursos/spam.PNG "Email: Respuesta de cuales son spam y cuales no")


La respuesta es...

![respuesta spam](https://raw.githubusercontent.com/dccuchile/CC6205/master/tutorials/recursos/spam_2.PNG "Email: Respuesta de cuales son spam y cuales no")


Pero, estos también pueden representar otros tipos de categorías...

![categorías de comunicados](https://raw.githubusercontent.com/dccuchile/CC6205/master/tutorials/recursos/emails_clases.PNG "Email: Cual es la clase del email")

--------------------------
¿Cómo habrán encontrado la película?

![¿Cómo encontraron la película?](https://raw.githubusercontent.com/dccuchile/CC6205/master/tutorials/recursos/limpiapiscinas.PNG "Email: ¿Cómo encontraron la película?")


Créditos a [OndaMedia](https://ondamedia.cl/). Películas y materíal audiovisual chileno gratis, muy recomendado!

-------------------


Entonces, 

### ¿Qué es la clasificación de texto?

La clasificación de texto consiste en tomar distintos textos y asignarles alguna clase. Dichas clases varían según la task que queramos resolver. Por ejemplo:

    - Detectar emails SPAM -> SPAM, NO SPAM
    - Reviews de Peliculas -> Buena, Mas o menos, Mala, Malísima, Brutalmente mala.
    - Análisis de sentimientos de tweets: Felicidad, Tristeza, Enojo, Ira,...
    - Detectar Fake News -> Es, No es
    - Lenguaje del texto -> Español, Inglés, Chino,...
    - Categoría de una noticia -> Nacional, Internacional, Economía, Sociedad, Opinión...

Se define formalmente como:

- Input: 

    - Un documento $d$
    - Un conjunto fijo de clases $c_1, c_2, ..., c_j$

- Output: 
    
    - Una clase $c \in C$ para el documento 
    
 
Hay dos clases de métodos para resolver estos problemas: 

1. **Hand-coded Rules 🤙**: 

Establecemos a mano las reglas que permiten detectar las clases.

2. **Supervised Machine Learning 💻**:
   
Entrenamos clasificadores a partir de muchos ejemplos de documentos etiquetados a mano. 

----------------------------------------


## ¿Qué haremos a continuación?


Cómo dijimos al comienzo, en este auxiliar crearemos un sistema que nos permita clasificar noticias de la radio biobio en 5 categorías: `nacional`, `internacional`, `economía`, `sociedad`, `opinion`.

Primero que nada, descargaremos los datos con los que trabajaremos.

Luego, generaremos el sistema mas básico, el cual consiste en transformar nuestro texto a `Bag of Words (BoW)` y luego, usar esos vectores para entrenar un clasificador. Este sistema nos puede entregar un muy buen baseline para comenzar a mejorar.

A continuación, veremos como mejorar aun mas nuestros resultados. Para esto agregaremos muchas mas técnicas vistas en cátedra, tales como el `preprocesamiento de texto`, `la creación de features personalizadas`, `reducción de dimensionalidad`, como también probar con clasificadores aún mas sofisticados.



### Cargar los datasets 


Los datos que usaremos son 5 conjuntos de noticias obtenidas de la radio biobio.
Cada categoría contiene 200 documentos (noticias). 

Los cargaremos directamente desde el github del curso utilizando la librería `pandas` 🐼: 


In [19]:
nacional = pd.read_json("https://github.com/dccuchile/CC6205/releases/download/Data/biobio_nacional.json", encoding ='utf-8')
internacional = pd.read_json("https://github.com/dccuchile/CC6205/releases/download/Data/biobio_internacional.json", encoding ='utf-8')
economia = pd.read_json("https://github.com/dccuchile/CC6205/releases/download/Data/biobio_economia.json", encoding ='utf-8')
sociedad = pd.read_json("https://github.com/dccuchile/CC6205/releases/download/Data/biobio_sociedad.json", encoding ='utf-8')
opinion = pd.read_json("https://github.com/dccuchile/CC6205/releases/download/Data/biobio_opinion.json", encoding ='utf-8')

In [26]:
dataset = pd.concat([nacional, internacional, economia, sociedad, opinion])
# Dejamos solo el titulo y el contenido y la clase
dataset = dataset[['title', 'content', 'category']]
# Ahora, mezclamos el titulo y el contenido, dejando así solo dos columnas:
dataset['content'] = dataset['title'] + dataset['content']
dataset = dataset.drop(columns=['title'])

Veamos unos cuantos ejemplos: 

In [30]:
dataset.sample(10)

,content,category
117,Reino Unido descarta un intercambio de petrole...,Internacional
153,Autoridad Palestina anuncia que dejará de resp...,Internacional
95,Sename anuncia supervisión extraordinaria para...,Nacional
74,Diputado Winter: No podemos seguir de brazos c...,Economia
94,Wifi en las casas de Cuba: la isla dio un paso...,Internacional
36,"Aseguran que director de Gendarmería sabía ""ha...",Nacional
156,Espacio Público y reunión de diputados con Pal...,Opinion
29,Quintero y Puchuncaví: estudio muestra que niñ...,Nacional
22,"""Ahí trabajan los retrasados"": burlas a trabaj...",Sociedad
97,China es principal destino: exportación de car...,Economia


Ahora, el procedimiento estandar de mineria de datos de kinder: **Dividir nuestros conjuntos en train y test.**

In [34]:
X_train, X_test, y_train, y_test = train_test_split(dataset.content,
                                                    dataset.category,
                                                    test_size=0.33,
                                                    random_state=42)

### Nuestro primer sistema de clasificación


Ahora que tenemos cargado el dataset, podemos implementar nuestro clasificador!

Para esto, usaremos 3 herramientas fundamentales de scikit-learn: un `pipeline`, `CountVectorizer` y `MultinomialNB`.

#### Pipeline


Un [`pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) es la definición de los procesos que llevará a cabo el sistema que creemos. Nos permite tener unificados todos los procesos a la vez que simplifica el código de nuestro sistema.


#### Bag of Words y CountVectorizer


¿Qué era Bag of Words?

Es un modelo en donde transformamos cada una de las oraciones de nuestro dataset en vectores. Cada vector contiene una columna por cada palabra / **token** del vocabulario. Al procesar el dataset, cada oración es mapeada a un vector que cuenta las apariciones de cada una de sus tokens. 

Referencia: [BoW en wikipedia](https://es.wikipedia.org/wiki/Modelo_bolsa_de_palabras)

**Un pequeño ejemplo**

Supongamos que nuestro tokenizador solo separa por espacios.

    - Doc1 : 'I love dogs'
    - Doc2: 'I hate dogs and knitting.
    - Doc3: 'Knitting is my hobby and my passion.

El bag of words quedaría:

<img src="https://i1.wp.com/datameetsmedia.com/wp-content/uploads/2017/05/bagofwords.004.jpeg" alt="BoW" style="width: 600px;"/>

`CountVectorizer` es la clase de `scikit` que transformará nuestro texto a Bag of Words. Fijense que es tremendamente útil tenerla dentro de un pipeline ya que fija en un comienzo el vocabulario que tendrá el Bag of Words, evitando discordancias entre los vectores del conjunto de entrenamiento y el de prueba.

#### MultinomialNB

Simplemente, el clásificador de Bayes.

TODO: Explicar un poquito mas esto...



------------------------


**Primero, definimos el pipeline**

In [35]:
# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer()  

# Definimos el clasificador que usaremos.
clf = MultinomialNB()   

# Creamos el pipeline
text_clf = Pipeline([('vect', vectorizer), ('clf', clf)])

**Luego, lo entrenamos**

In [36]:
# Entrenamos nuestro pipeline
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

**Y predecimos**

In [37]:
y_pred = text_clf.predict(X_test)

**Veamos como nos fue:**


In [53]:
# algunos ejemplos:
pd.DataFrame({'content': X_test, 'category':y_test, 'predicted category': y_pred}).sample(10)

,content,category,predicted category
25,Ranking de las 34 mejores empresas para jóvene...,Economia,Economia
72,Caso Línea Azul: parlamentarios plantean imple...,Nacional,Nacional
59,Servicios Sanitarios niega lobby ante crisis d...,Nacional,Nacional
54,Revés para Boris Johnson en su primera prueba ...,Internacional,Internacional
167,Adolescente logra atrapar en el aire a niña de...,Sociedad,Sociedad
181,Accidente en Neuquén: familiares de chilena fa...,Internacional,Nacional
162,Corea del Norte dispara dos misiles de corto a...,Internacional,Internacional
137,"A mitad de agosto se realizará nuevamente ""Rut...",Nacional,Opinion
39,A 20 aumentan muertos por tiroteo en centro co...,Internacional,Internacional
161,Nuevo primer ministro Boris Johnson elige gabi...,Internacional,Internacional


In [54]:
# usando la matriz de confusión:
confusion_matrix(y_test, y_pred)

array([[63,  0,  3,  4,  0],
       [ 2, 68,  2,  2,  3],
       [ 2,  0, 40, 15,  1],
       [ 4,  1,  0, 61,  0],
       [ 1,  7,  0,  2, 49]], dtype=int64)

Recordatorio: 
TODO: Mejorar esto,.,,
- `precision` is the number of correct results divided by the number of all returned results. 
- `recall` is the number of correct results divided by the number of results that should have been returned

In [55]:
# usando el classification report:
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

     Economia       0.88      0.90      0.89        70
Internacional       0.89      0.88      0.89        77
     Nacional       0.89      0.69      0.78        58
      Opinion       0.73      0.92      0.81        66
     Sociedad       0.92      0.83      0.88        59

     accuracy                           0.85       330
    macro avg       0.86      0.85      0.85       330
 weighted avg       0.86      0.85      0.85       330



-------------------------

Se ven bastante buenos los resultados. ¿Pero, podremos mejorarlos?

### Preprocesamiento del texto

En clases vimos que habían varias técnicas que permiter preprocesar los textos.
Es decir, cómo hacemos el proceso de tokenización (separación de las palabras).



Alguna de las técnicas son:


- Eliminación de Stopwords
- Stemming
- Lematización

Existen otros preprocesadores que agregan información a las oraciones, tales como aquellos que indican negaciones.

A continuación, describiremos con mas detalle cada uno de estas técnicas.

#### Tokenizar 🔪

¿Qué era tokenizar?


    Es el proceso de convertir una secuencia de carácteres (por ejemplo, una oración) en una secuencia de valores distintos entre si llamados tokens.
    
Referencia: [Tokenización en wikipedia](https://en.wikipedia.org/wiki/Lexical_analysis#Tokenization)



##### spaCy y el objeto nlp

`nlp` es el objeto que nos permite usar e interactuar con la librería [`spacy`](https://spacy.io/).
Esta librería incluye variadas herramientras, tales como tokenizar, lematizar, descartar stopwords, entre otras (para este auxiliar, solo utilizaremos las mencionadas). El objeto nlp lo instanciamos en la sección de imports.

Para usarla, simplemente se le pasa el texto como parámetro, como veremos en el siguiente ejemplo: 

In [79]:
DOC = "quiero salir de mi casa!! esto no es un meme. auxilio..."

tokens = []
for word in nlp(DOC):
    tokens.append(word)

tokens

[quiero, salir, de, mi, casa, !, !, esto, no, es, un, meme, ., auxilio, ...]

**Observación**: Para este auxiliar usaremos `List Comprehension`, otra forma de hacer un for un poco mas reducida.
Una muy buena referencia de esto [aquí](https://www.programiz.com/python-programming/list-comprehension).

La operación anterior usando esta sintáxis quedaría como:

In [80]:
tokens = [word for word in nlp(DOC)]
tokens

[quiero, salir, de, mi, casa, !, !, esto, no, es, un, meme, ., auxilio, ...]

#### Stopwords 🛑

¿Qué eran las stopwords?

    Las Stopwords son palabras muy comunes en nuestro lenguaje y que por lo tanto, no aportan mucha información. Existen múltiples listas de stopwords para muchos idiomas y la aplicación de estas variará caso a caso.

    
Referencias: [Stopwords en Wikipedia](https://en.wikipedia.org/wiki/Stop_words)

En este caso, utilizaremos las stopwords inlcuidas en la librería spaCy en español

In [76]:
print(len(STOP_WORDS))
print(list(STOP_WORDS)[0:20])

551
['esas', 'siete', 'unas', 'diferente', 'muy', 'total', 'varias', 'dan', 'allí', 'aunque', 'cierto', 'repente', 'primeros', 'sino', 'sois', 'claro', 'era', 'sí', 'considera', 'dias']


#### Stemming

¿Qué era el stemming? 

    Son un conjunto de métodos enfocados en reducir cada palabra a su raiz.

Referencia: [Stemming en Wikipedia](https://en.wikipedia.org/wiki/Stemming)
  
#### Ejemplos: 


| word | stem of the word  |
|---|---|
working | work
worked | work
works | work

#### nltk

En este caso, utilizaremos la segunda librería de herramientas de nlp: [`nltk`](https://www.nltk.org/). Esta provee una buena herramienta para hacer stemming en español : `SnowballStemmer`

In [97]:
stemmer = SnowballStemmer('spanish')
stemmed_doc = [stemmer.stem(str(token)) for token in tokens]
print(stemmed_doc)

['quier', 'sal', 'de', 'mi', 'cas', '!', '!', 'esto', 'no', 'es', 'un', 'mem', '.', 'auxili', '...']


### Lematización 🙀

¿Qué era lematización? 

    Es el proceso de transformar cada token a su lema, el cual es la palabra base sin ningún tipo de flexión (o alteración)
    
    

  
    
    
Referencia: [Lematización en wikipedia](https://en.wikipedia.org/wiki/Lemmatisation)
    
#### Ejemplos

| word | lemma  |
|---|---|
dije| decir 
guapas | guapo
mesa | mesas


 <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/29/Flexi%C3%B3nGato-svg.svg/300px-Flexi%C3%B3nGato-svg.svg.png" alt="Flexión de gato" style="width: 200px;"/>
 

#### Lematizar el texto

Al igual que la tokenización, utilizaremos `scpaCy` (a través del objeto `nlp`) para lematizar el contenido.

In [107]:
lemmatized_content = [word.lemma_ for word in nlp(DOC)]
print(lemmatized_content)

['querer', 'salir', 'de', 'mi', 'casar', '!', '!', 'este', 'no', 'ser', 'uno', 'meme', '.', 'auxiliar', '...']



**Discusión:**

    ¿Cuál es mejor?

### Agregar nuestros tokenizadores al sistema

Para agregar los tokenizadores en el sistema, los envolvemos en funciones que procesan independientemente cada texto. Luego, `CountVectorizer` se encargará de procesar cada documento del dataset.

#### Tokenizadores para CountVectorizer

In [119]:
# Tokenizers para CountVectorizer

# Solo tokenizar el doc usando spacy.
def tokenizer(doc):
    return [x.orth_ for x in nlp(doc)]


# Tokenizar y remover las stopwords del doc
def tokenizer_with_stopwords(doc):
    return [x.orth_ for x in nlp(doc) if x.orth_ not in STOP_WORDS]


# Tokenizar y lematizar.
def tokenizer_with_lemmatization(doc):
    return [x.lemma_ for x in nlp(doc)]

# Tokenizar y hacer stemming.
def tokenizer_with_stemming(doc):
    stemmer = SnowballStemmer('spanish')
    return [stemmer.stem(word) for word in [x.orth_ for x in nlp(doc)]]

#### Creamos nuestro nuevo pipeline

In [124]:
# seleccionamos el tokenizador a usar:
TOKENIZER = tokenizer_with_lemmatization

# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer(analyzer='word',
                             tokenizer=TOKENIZER,
                             ngram_range=(1, 1))

# Definimos el clasificador que usaremos.
clf = MultinomialNB()   

# Creamos el pipeline
text_clf_2 = Pipeline([('vect', vectorizer), ('clf', clf)])

#### Entrenamos nuestro pipeline y predecimos


In [125]:
text_clf_2.fit(X_train, y_train)
y_pred = text_clf_2.predict(X_test)

#### Evaluamos

In [126]:
# usando la matriz de confusión:
print(confusion_matrix(y_test, y_pred),
      '\n\n-------------------------------------------------------\n')
# usando el classification report:
print(classification_report(y_test, y_pred))

[[62  0  3  5  0]
 [ 2 68  2  3  2]
 [ 1  0 43 13  1]
 [ 4  1  0 61  0]
 [ 1  6  0  4 48]] 

-------------------------------------------------------

               precision    recall  f1-score   support

     Economia       0.89      0.89      0.89        70
Internacional       0.91      0.88      0.89        77
     Nacional       0.90      0.74      0.81        58
      Opinion       0.71      0.92      0.80        66
     Sociedad       0.94      0.81      0.87        59

     accuracy                           0.85       330
    macro avg       0.87      0.85      0.85       330
 weighted avg       0.87      0.85      0.86       330



# ------------------------------ WIP --------------------------------

### Clasificación de tópicos de noticias con Naive Bayes


¿Qué es?

    In machine learning, naive Bayes classifiers are a family of simple "probabilistic classifiers" based on applying Bayes' theorem with strong (naive) independence assumptions between the features. 

- Simple (“naïve”) classification method based on Bayes rule that relies on very simple representation of document: *Bag of words*:

Given a problem instance to be classified, represented by a vector $x =(x_{1},\dots ,x_{n})$ representing some n features (independent variables), it assigns to this instance probabilities:

$$ p(C_k | x_1, \dots, x_n) $$

or each of $K$ possible outcomes or classes $ C_{k}$.


Using Bayes' theorem, the conditional probability can be decomposed as 

$$ p(C_k | x ) = \frac{p (C_k) p(x | C_k)}{p(x)}$$

In plain english:

$$posterior = \frac{prior * likehood}{evidence} $$


#### En este caso...

$$ p(\ nacional\ |\  [0,0,3,0,6,0,2,\dots] ) = \frac{p (\ nacional\ )\ p([0,0,3,0,6,0,2,\dots]\ | \ nacional)}{p([0,0,3,0,6,0,2,\dots])}$$


Referencia : [Naive Bayes Classifier](https://en.wikipedia.org/wiki/Naive_Bayes_classifier)

Observación: Dado el supuesto de independencia del clasificador, no se deben utilizar n-gramas.


In [ ]:
predicted = text_clf.predict(X_test)

conf = confusion_matrix(y_test, predicted)
score = f1_score(y_test, predicted, average='macro') 
class_rep = classification_report(y_test, predicted)

print('\nConfusion Matrix for Logistic Regression + ngram features:')
print(conf)
print('\nClassification Report')
print(class_rep)
print('\nF1:'+str(score))


#### Ejemplos

In [130]:
text_clf.predict(["En puerto montt se encontró un perrito, que aparentemente, habría consumido drogas de alto calibre. Producto de esto, padecera severa caña durante varios dias."])

array(['Sociedad'], dtype='<U13')

In [129]:
text_clf.predict(["kim jong un será el próximo candidato a ministro de educación."])

array(['Internacional'], dtype='<U13')

In [128]:
text_clf.predict(["El banco mundial presentó para chile un decrecimiento económico de 92% y una inflación de 8239832983289%."])

array(['Economia'], dtype='<U13')

### Clasificación de tópicos de noticias con Regresión Logísitica

No profundizaremos en este clasificador, mas del hecho de que se "supone" que debería tener mejor rendimiento que el de bayes.

Referencia: [Regresión Logística](https://en.wikipedia.org/wiki/Logistic_regression)

#### Establecer el Pipeline

In [ ]:
# Qué tokenizer usaremos?
TOKENIZER = tokenizer_with_lemmatization

log_mod = LogisticRegression(solver='lbfgs', multi_class='ovr', max_iter = 1000)   
log_pipe = Pipeline([('vect', vectorizer), ('clf', log_mod)])

#### Entrenar

In [ ]:
log_pipe.fit(X_train, y_train)

#### Evaluar

In [ ]:
predicted = log_pipe.predict(X_test)

conf = confusion_matrix(y_test, predicted)
score = f1_score(y_test, predicted, average='macro') 
class_rep = classification_report(y_test, predicted)

print('\nConfusion Matrix for Logistic Regression + ngram features:')
print(conf)
print('\nClassification Report')
print(class_rep)
print('\nF1 Score: '+str(score))

#### Ejemplos

In [ ]:
log_pipe.predict(["En puerto montt se encontró un perrito, que aparentemente, habría consumido drogas de alto calibre. Producto de esto, padecera severa caña durante varios dias."])

In [ ]:
log_pipe.predict(["kim jong un será el próximo candidato a ministro de educación."])

### Clasificación de Autoría de documentos

¿Existirá un patrón en como escriben los periodistas que nos permitan identificarlos a partir de sus textos?

In [ ]:
pd.concat(datasets).author.unique()

In [ ]:
def process_datasets_by_author(datasets):
    dataset = pd.concat(datasets)
    X_train, X_test, y_train, y_test = train_test_split(dataset.content, dataset.author, test_size=0.33, random_state=42)
    
    return X_train, X_test, y_train, y_test


In [ ]:
X_train_2, X_test_2, y_train_2, y_test_2 = process_datasets_by_author(datasets)

#### Definir el Pipeline

In [ ]:
# Qué tokenizer usaremos?
TOKENIZER = tokenizer_with_lemmatization

log_mod_by_author = LogisticRegression(solver='lbfgs', multi_class='ovr', max_iter = 1000)   
log_pipe_by_author = Pipeline([('vect', vectorizer), ('clf', log_mod_by_author)])

#### Entrenar

In [ ]:
log_pipe_by_author.fit(X_train_2, y_train_2)

#### Evaluar

In [ ]:
predicted = log_pipe_by_author.predict(X_test_2)

conf = confusion_matrix(y_test_2, predicted)
score = f1_score(y_test_2, predicted, average='macro') 
class_rep = classification_report(y_test_2, predicted)

print('\nConfusion Matrix for Logistic Regression + ngram features:')
print(conf)
print('\nClassification Report')
print(class_rep)
print('\nF1 Score:'+str(score))

## Créditos

Todas las noticias extraidas perteneces a [Biobio Chile](https://www.biobiochile.cl/), los cuales gentilmente licencian todo su material a través de la [licencia Creative Commons (CC-BY-NC)](https://creativecommons.org/licenses/by-nc/2.0/cl/)

## Referencias

Gitgub del curso: 
- https://github.com/dccuchile/CC6205

Slides:
- https://web.stanford.edu/~jurafsky/slp3/slides/7_NB.pdf


Análisis de sentimientos como clasificación de texto:
- https://affectivetweets.cms.waikato.ac.nz/benchmark/

Algunos Recursos útiles
- [Pandas Cheat Sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)
- [Scikit-learn Cheat Sheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Scikit_Learn_Cheat_Sheet_Python.pdf)
- [Spacy Tutorial](https://www.datacamp.com/community/blog/spacy-cheatsheet)
- [NLTK Cheat sheet](http://sapir.psych.wisc.edu/programming_for_psychologists/cheat_sheets/Text-Analysis-with-NLTK-Cheatsheet.pdf)